In [1]:
import osmnx as ox
from Facility import *
import networkx as nx
from constants import *

In [2]:
class Advertisement:
    def __init__(self):
        self.G=ox.graph_from_bbox(north, south, east, west, network_type='drive')
        self.G=nx.convert_node_labels_to_integers(self.G) 
        self.all_pair_short()
        self.add_structure() 
    def all_pair_short(self):
        self.paths=[]
        for i in self.G.nodes:
            self.paths.append(nx.single_source_dijkstra_path_length(self.G,i,weight='length',cutoff=50000))
    def add_structure(self):
        for u,v,n in self.G.edges:
            self.G.edges[u,v,n]['Facilities']=[]
            self.G.edges[u,v,n]['relevant']={}
            self.G.edges[u,v,n]['relevantR']={}
    def add_facility(self,u,v,n,ident,R):
        if not self.G.has_edge(u,v,n):
            return
        self.G.edges[u,v,n]['Facilities'].append(Facility(ident,R))
        self.G.edges[u,v,n]['Facilities'].sort(key=lambda x: x.R, reverse=True)
             
        if len(self.G.edges[u,v,n]['Facilities'])==1:
            self.preprocessR(u,v,n)
        if R<self.G.edges[u,v,n]['Facilities'][0].R:
            return    
        self.preprocess(u,v,n)    
    
    #def get_key(obj):
    #    return obj.R
    def modify(self,ide,u,v,n,newR):
        facility=self.G.edges[u,v,n]['Facilities']
        Rad=-1
        for i in facility:        
            if i.ident==ide:
                Rad=facility[0].R
                i.R=newR
                break
        facility.sort(key=lambda x:x.R,reverse=True) 
        newRad=facility[0].R
        if(Rad==-1):
            print("Facility do not exist")
            return
        if newRad>Rad:
            self.preprocess(u,v,n)
        elif newRad<Rad:
            self.preprocessd(newRad,Rad,u,v,n)
            
    def preprocessd(self,Lbound,Ubound,u,v,n):
        RTD=0
        for w in self.paths[v]:
            for z in self.G.adj[w]:
                for n1 in self.G.adj[w][z]:
                    RTD=self.G.edges[w,z,n1]['length']+self.G.edges[u,v,n]['length']+self.paths[v][w]
                    if u in self.paths[z].keys():
                        RTD+=self.paths[z][v]
                        if RTD>Lbound and RTD<=Ubound:
                            del self.G.edges[w,z,n1]['relevant'][(u,v,n)]
                            self.G.edges[w,z,n1]['relevantR'].update({(u,v,n):RTD})
    def delete_facility(self,u,v,n,ide):
        facility=self.G.edges[u,v,n]['Facilities']
        Rad=facility[0].R
        for i in facility:
            if i.ident==ide:
                if len(facility)==1:
                    self.preprocessd(0,facility[0].R,u,v,n)
                    self.preprocessRDel(u,v,n)
                    facility.clear()
                else:
                    facility.remove(i)
                    facility.sort(key= lambda x:x.R,reverse=True)
                    if(Rad>facility[0].R):
                        self.preprocessd(facility[0].R,Rad,u,v,n)    
                break  
        
    def preprocessRDel(self,u,v,n):
        RTD=0
        Re=50000
        for w in self.paths[v]:
            for z in self.G.adj[w]:
                for n1 in self.G.adj[w][z]:
                    RTD=self.G.edges[w,z,n1]['length']+self.G.edges[u,v,n]['length']+self.paths[v][w]
                    if u in self.paths[z].keys():
                        RTD+=self.paths[z][v]
                        if RTD<=Re: 
                            del self.G.edges[w,z,n1]['relevantR'][(u,v,n)]  
    def preprocessR(self,u,v,n):
        RTD=0
        Re=50000
        for w in self.paths[v]:
            for z in self.G.adj[w]:
                for n1 in self.G.adj[w][z]:
                    RTD=self.G.edges[w,z,n1]['length']+self.G.edges[u,v,n]['length']+self.paths[v][w]
                    if u in self.paths[z].keys():
                        RTD+=self.paths[z][v]
                        if RTD<=Re and (u,v,n) not in self.G.edges[w,z,n1]['relevantR'].keys():
                            self.G.edges[w,z,n1]['relevantR'].update({(u,v,n):RTD})
                            
                            
    def preprocess(self,u,v,n):
        RTD=0
        Re=self.G.adj[u][v][n]['Facilities'][0].R
        for w in self.paths[v]:
            for z in self.G.adj[w]:
                for n1 in self.G.adj[w][z]:
                    RTD=self.G.edges[w,z,n1]['length']+self.G.edges[u,v,n]['length']+self.paths[v][w]
                    if u in self.paths[z].keys():
                        RTD+=self.paths[z][v]
                        if RTD<=Re and (u,v,n) not in self.G.edges[w,z,n1]['relevant'].keys():
                            self.G.edges[w,z,n1]['relevant'].update({(u,v,n):RTD})
                            del self.G.edges[w,z,n1]['relevantR'][(u,v,n)]
                            
    
    def query(self,u,v,n):
        facility=[]
        for w,z,n1 in self.G.edges[u,v,n]['relevant'].keys():
            distance= self.G.edges[u,v,n]['relevant'][(w,z,n1)]
            facility.append(((w,z,n1),distance))
        facility.sort(key=lambda x: x[1])
        for i,d in facility:
            for f in self.G.adj[i[0]][i[1]][i[2]]['Facilities']:
                if d>f.R:
                    break
                print(f.R,d)       
        
            
    def queryR(self,u,v,n,Range):
        facility=[]
        for w,z,n1 in self.G.edges[u,v,n]['relevant'].keys():
            distance= self.G.edges[u,v,n]['relevant'][(w,z,n1)]
            if distance>Range:
                continue
            facility.append(((w,z,n1),distance)) 
                            
        for w,z,n1 in self.G.edges[u,v,n]['relevantR'].keys():
            distance= self.G.edges[u,v,n]['relevantR'][(w,z,n1)]
            if distance>Range:
                continue
            facility.append(((w,z,n1),distance))
        
        facility.sort(key=lambda x: x[1])
        for i,d in facility:                                          
            for i in self.G.adj[i[0]][i[1]][i[2]]['Facilities']:            
                print(d) 
         
         
            

In [3]:
Obj=Advertisement() 

In [4]:
Obj.add_facility(0,3653,0,1,8000)
Obj.add_facility(0,3653,0,2,9000)
Obj.add_facility(0,3653,0,3,10000)
Obj.add_facility(2,115,0,4,8000)
Obj.add_facility(2,115,0,5,13000)
Obj.add_facility(0,7,0,6,2000)

In [10]:
#Obj.delete_facility(0,3653,0,3)
Obj.delete_facility(0,3653,0,1)

In [ ]:
Obj.modify(1,0,3653,0,8000)
Obj.modify(2,0,3653,0,9000)
Obj.modify(3,0,3653,0,10000)

In [ ]:
Obj.modify(3,0,3653,0,13000)

In [ ]:
Obj.G.edges[0,3653,0]

In [11]:
for u,v,n in Obj.G.edges:
    #if len(Obj.G.edges[u,v,n]['relevantR'])>0:
    print(u,v,n,Obj.G.edges[u,v,n]['relevantR'])
    print(u,v,n,Obj.G.edges[u,v,n]['relevant'])

0 3653 0 {(0, 7, 0): 15142.091}
0 3653 0 {(2, 115, 0): 8832.9}
0 7 0 {(0, 7, 0): 8833.279}
0 7 0 {(2, 115, 0): 10373.208}
2 115 0 {(0, 7, 0): 8632.568000000001}
2 115 0 {(2, 115, 0): 1624.411}
3 2 0 {(0, 7, 0): 10269.979000000003}
3 2 0 {(2, 115, 0): 5682.835}
4 2887 0 {}
4 2887 0 {}
4 2889 0 {}
4 2889 0 {}
5 3351 0 {}
5 3351 0 {}
6 2828 0 {(0, 7, 0): 11378.836000000001}
6 2828 0 {(2, 115, 0): 6210.545}
6 2923 0 {(0, 7, 0): 11982.496000000001}
6 2923 0 {(2, 115, 0): 5604.177}
7 8 0 {(0, 7, 0): 6315.173000000001}
7 8 0 {(2, 115, 0): 7870.012000000001}
8 9 0 {(0, 7, 0): 6315.173000000001}
8 9 0 {(2, 115, 0): 7870.012000000001}
9 2423 0 {(0, 7, 0): 6315.173000000001}
9 2423 0 {(2, 115, 0): 7870.012000000001}
9 2466 0 {(2, 115, 0): 17699.602000000006, (0, 7, 0): 16137.144}
9 2466 0 {}
10 2884 0 {(0, 7, 0): 11728.201000000001}
10 2884 0 {(2, 115, 0): 4665.768}
10 19 0 {(0, 7, 0): 13953.431}
10 19 0 {(2, 115, 0): 6976.062}
11 2892 0 {(0, 7, 0): 8695.947}
11 2892 0 {(2, 115, 0): 1624.41099999

81 2541 0 {(0, 7, 0): 11604.788}
81 2541 0 {(2, 115, 0): 11790.036999999998}
82 3718 0 {(0, 7, 0): 8833.279}
82 3718 0 {(2, 115, 0): 8731.089}
83 16 0 {}
83 16 0 {}
83 3654 0 {(0, 7, 0): 6315.173000000001}
83 3654 0 {(2, 115, 0): 7994.285}
84 472 0 {(2, 115, 0): 19209.605999999992, (0, 7, 0): 19066.247}
84 472 0 {}
84 2072 0 {(2, 115, 0): 19443.461999999996, (0, 7, 0): 19194.089999999997}
84 2072 0 {}
85 869 0 {(0, 7, 0): 6707.845}
85 869 0 {(2, 115, 0): 6601.209}
85 939 0 {(0, 7, 0): 6744.888000000001}
85 939 0 {(2, 115, 0): 6638.252}
85 2370 0 {(0, 7, 0): 6612.169}
85 2370 0 {(2, 115, 0): 6505.532999999999}
86 2387 0 {(0, 7, 0): 6897.9349999999995}
86 2387 0 {(2, 115, 0): 5547.298}
86 2390 0 {(0, 7, 0): 6682.387}
86 2390 0 {(2, 115, 0): 5675.821}
86 2805 0 {(0, 7, 0): 6481.053}
86 2805 0 {(2, 115, 0): 6223.959999999999}
86 3422 0 {(0, 7, 0): 6695.795}
86 3422 0 {(2, 115, 0): 5762.039999999999}
87 3657 0 {(0, 7, 0): 7056.421}
87 3657 0 {(2, 115, 0): 6510.135999999999}
87 678 0 {(0, 7,

172 3149 0 {(0, 7, 0): 8211.380000000001}
172 3149 0 {(2, 115, 0): 4297.155}
172 171 0 {(0, 7, 0): 8459.55}
172 171 0 {(2, 115, 0): 4510.439}
173 172 0 {(0, 7, 0): 8213.083999999999}
173 172 0 {(2, 115, 0): 4297.155}
173 169 0 {(0, 7, 0): 8075.718000000001}
173 169 0 {(2, 115, 0): 4120.026}
174 786 0 {(2, 115, 0): 14891.894999999997, (0, 7, 0): 14692.004}
174 786 0 {}
175 2714 0 {(0, 7, 0): 10496.597000000002}
175 2714 0 {(2, 115, 0): 10481.243999999999}
175 345 0 {(0, 7, 0): 10498.602}
175 345 0 {(2, 115, 0): 10483.249}
175 2836 0 {(0, 7, 0): 10309.898000000001}
175 2836 0 {(2, 115, 0): 10294.544999999998}
176 3501 0 {(2, 115, 0): 13304.609999999997, (0, 7, 0): 18445.606}
176 3501 0 {}
176 620 0 {(2, 115, 0): 13405.915999999997, (0, 7, 0): 18546.911999999997}
176 620 0 {}
177 2292 0 {(0, 7, 0): 17050.237}
177 2292 0 {(2, 115, 0): 11825.355}
177 22 0 {(2, 115, 0): 13759.701000000001, (0, 7, 0): 18941.366}
177 22 0 {}
177 1101 0 {(0, 7, 0): 17100.686}
177 1101 0 {(2, 115, 0): 11919.021}

232 1518 0 {(2, 115, 0): 4459.066}
232 333 0 {(0, 7, 0): 6632.774000000001}
232 333 0 {(2, 115, 0): 4288.2}
232 2308 0 {(0, 7, 0): 6592.594999999999}
232 2308 0 {(2, 115, 0): 4450.858}
232 231 0 {(0, 7, 0): 6430.6320000000005}
232 231 0 {(2, 115, 0): 4287.576}
233 1019 0 {(0, 7, 0): 6431.383000000001}
233 1019 0 {(2, 115, 0): 4781.205}
233 234 0 {(0, 7, 0): 6595.512000000001}
233 234 0 {(2, 115, 0): 4774.793000000001}
233 2308 0 {(0, 7, 0): 6430.652}
233 2308 0 {(2, 115, 0): 4609.933}
233 334 0 {(0, 7, 0): 6633.554000000001}
233 334 0 {(2, 115, 0): 4609.9980000000005}
234 2074 0 {(0, 7, 0): 6431.75}
234 2074 0 {(2, 115, 0): 4945.815}
234 2770 0 {(0, 7, 0): 6596.448}
234 2770 0 {(2, 115, 0): 4940.055}
234 233 0 {(0, 7, 0): 6431.186}
234 233 0 {(2, 115, 0): 4774.793000000001}
235 3475 0 {(0, 7, 0): 7231.661000000001}
235 3475 0 {(2, 115, 0): 5742.374999999999}
235 931 0 {(0, 7, 0): 7231.438000000001}
235 931 0 {(2, 115, 0): 5844.264999999999}
235 236 0 {(0, 7, 0): 7636.872000000001}
235 

280 2644 0 {(2, 115, 0): 6298.428}
281 362 0 {(0, 7, 0): 12472.269}
281 362 0 {(2, 115, 0): 7565.804000000001}
281 266 0 {(0, 7, 0): 12428.314}
281 266 0 {(2, 115, 0): 7521.849}
281 282 0 {(0, 7, 0): 12825.615000000002}
281 282 0 {(2, 115, 0): 7919.15}
281 2587 0 {(0, 7, 0): 12427.479000000001}
281 2587 0 {(2, 115, 0): 7521.014000000001}
282 1775 0 {(0, 7, 0): 12870.325}
282 1775 0 {(2, 115, 0): 7963.860000000001}
282 267 0 {(0, 7, 0): 12826.608}
282 267 0 {(2, 115, 0): 7920.143}
282 281 0 {(0, 7, 0): 12825.615000000002}
282 281 0 {(2, 115, 0): 7919.1500000000015}
282 283 0 {(0, 7, 0): 13227.118000000002}
282 283 0 {(2, 115, 0): 8320.653}
283 1022 0 {(0, 7, 0): 13272.862000000001}
283 1022 0 {(2, 115, 0): 8366.397}
283 268 0 {(0, 7, 0): 13227.118000000002}
283 268 0 {(2, 115, 0): 8320.653000000002}
283 284 0 {(0, 7, 0): 13629.29}
283 284 0 {(2, 115, 0): 8722.825}
283 282 0 {(0, 7, 0): 13227.563000000002}
283 282 0 {(2, 115, 0): 8321.098}
284 1821 0 {(0, 7, 0): 13673.404000000002}
284 1

...

3459 3443 0 {(2, 115, 0): 8952.217999999999}
3459 993 0 {(0, 7, 0): 14203.974}
3459 993 0 {(2, 115, 0): 9287.579999999998}
3460 1426 0 {(0, 7, 0): 9506.905000000002}
3460 1426 0 {(2, 115, 0): 8759.550000000001}
3461 1333 0 {(0, 7, 0): 9919.777000000002}
3461 1333 0 {(2, 115, 0): 9139.241000000002}
3462 3275 0 {(0, 7, 0): 9160.776000000002}
3462 3275 0 {(2, 115, 0): 6230.4490000000005}
3463 2458 0 {(2, 115, 0): 16965.147999999994, (0, 7, 0): 18830.164000000004}
3463 2458 0 {}
3463 2007 0 {(2, 115, 0): 17237.841999999997, (0, 7, 0): 18699.982000000004}
3463 2007 0 {}
3463 316 0 {(2, 115, 0): 17043.531999999996, (0, 7, 0): 18908.548000000003}
3463 316 0 {}
3463 315 0 {(2, 115, 0): 16834.965999999993, (0, 7, 0): 18700.151000000005}
3463 315 0 {}
3464 951 0 {(2, 115, 0): 19568.243, (0, 7, 0): 19479.291000000005}
3464 951 0 {}
3464 1258 0 {(2, 115, 0): 19296.772999999997, (0, 7, 0): 19207.821000000004}
3464 1258 0 {}
3464 2655 0 {(2, 115, 0): 19690.189, (0, 7, 0): 19601.237000000005}
3464 26

3524 1117 0 {(2, 115, 0): 10193.302000000001}
3524 1117 1 {(0, 7, 0): 15109.696000000004}
3524 1117 1 {(2, 115, 0): 10193.302000000001}
3525 1117 0 {(0, 7, 0): 15084.694000000003}
3525 1117 0 {(2, 115, 0): 10168.300000000001}
3525 1117 1 {(0, 7, 0): 15084.694000000003}
3525 1117 1 {(2, 115, 0): 10168.300000000001}
3525 3695 0 {(0, 7, 0): 15058.182}
3525 3695 0 {(2, 115, 0): 10141.788}
3525 3695 1 {(0, 7, 0): 15058.182}
3525 3695 1 {(2, 115, 0): 10141.788}
3526 3528 0 {(0, 7, 0): 14706.820000000003}
3526 3528 0 {(2, 115, 0): 9790.426}
3526 3528 1 {(0, 7, 0): 14706.820000000003}
3526 3528 1 {(2, 115, 0): 9790.426}
3526 1827 0 {(0, 7, 0): 14696.902000000002}
3526 1827 0 {(2, 115, 0): 9780.508}
3526 1827 1 {(0, 7, 0): 14696.902000000002}
3526 1827 1 {(2, 115, 0): 9780.508}
3527 3702 0 {(0, 7, 0): 14866.554000000004}
3527 3702 0 {(2, 115, 0): 9950.16}
3527 3702 1 {(0, 7, 0): 14866.554000000004}
3527 3702 1 {(2, 115, 0): 9950.16}
3527 3528 0 {(0, 7, 0): 14738.556000000002}
3527 3528 0 {(2, 1

3568 3567 0 {(2, 115, 0): 9622.768}
3568 796 0 {(0, 7, 0): 9872.351999999999}
3568 796 0 {(2, 115, 0): 10024.43}
3568 3572 0 {(0, 7, 0): 9565.261}
3568 3572 0 {(2, 115, 0): 9622.378}
3569 3570 0 {(0, 7, 0): 9667.494999999999}
3569 3570 0 {(2, 115, 0): 9822.326999999997}
3569 3547 0 {(0, 7, 0): 9652.579}
3569 3547 0 {(2, 115, 0): 9933.793}
3569 2909 0 {(0, 7, 0): 9647.815999999999}
3569 2909 0 {(2, 115, 0): 9803.038}
3570 3569 0 {(0, 7, 0): 9667.936}
3570 3569 0 {(2, 115, 0): 9823.158}
3570 795 0 {(0, 7, 0): 10073.123}
3570 795 0 {(2, 115, 0): 10226.921999999999}
3570 3568 0 {(0, 7, 0): 9667.495}
3570 3568 0 {(2, 115, 0): 9822.327}
3571 3572 0 {(0, 7, 0): 9361.001}
3571 3572 0 {(2, 115, 0): 9418.118}
3571 3567 0 {(0, 7, 0): 9449.199}
3571 3567 0 {(2, 115, 0): 9604.420999999998}
3572 3571 0 {(0, 7, 0): 9381.975000000002}
3572 3571 0 {(2, 115, 0): 9439.092}
3572 797 0 {(0, 7, 0): 9763.59}
3572 797 0 {(2, 115, 0): 9820.706999999999}
3572 1122 0 {(0, 7, 0): 9361.001}
3572 1122 0 {(2, 115, 0

3628 1854 0 {(2, 115, 0): 7605.586}
3628 1350 0 {(0, 7, 0): 8217.697}
3628 1350 0 {(2, 115, 0): 7886.964}
3628 3627 0 {(0, 7, 0): 8141.706}
3628 3627 0 {(2, 115, 0): 7810.973}
3629 3630 0 {(0, 7, 0): 7696.5419999999995}
3629 3630 0 {(2, 115, 0): 7365.808999999999}
3629 3631 0 {(0, 7, 0): 7678.527999999998}
3629 3631 0 {(2, 115, 0): 7347.795}
3630 3629 0 {(0, 7, 0): 7678.528}
3630 3629 0 {(2, 115, 0): 7347.795}
3630 1349 0 {(0, 7, 0): 7959.504000000001}
3630 1349 0 {(2, 115, 0): 7628.771}
3630 3628 0 {(0, 7, 0): 7936.3189999999995}
3630 3628 0 {(2, 115, 0): 7605.586}
3631 3632 0 {(0, 7, 0): 7425.347}
3631 3632 0 {(2, 115, 0): 7094.614}
3631 1908 0 {(0, 7, 0): 7407.085000000001}
3631 1908 0 {(2, 115, 0): 7076.351999999999}
3632 3631 0 {(0, 7, 0): 7407.084999999999}
3632 3631 0 {(2, 115, 0): 7076.352}
3632 990 0 {(0, 7, 0): 7498.5869999999995}
3632 990 0 {(2, 115, 0): 7352.227999999999}
3632 3630 0 {(0, 7, 0): 7678.528}
3632 3630 0 {(2, 115, 0): 7347.795}
3633 1908 0 {(0, 7, 0): 7128.811}

3695 3694 1 {(0, 7, 0): 15019.238000000001}
3695 3694 1 {(2, 115, 0): 10102.844000000001}
3696 1599 0 {(0, 7, 0): 7582.793000000001}
3696 1599 0 {(2, 115, 0): 7568.237999999999}
3696 3124 0 {(0, 7, 0): 7997.485000000001}
3696 3124 0 {(2, 115, 0): 7982.93}
3696 3697 0 {(0, 7, 0): 7646.869000000001}
3696 3697 0 {(2, 115, 0): 7632.314}
3697 3696 0 {(0, 7, 0): 7646.869000000001}
3697 3696 0 {(2, 115, 0): 7632.314}
3698 161 0 {(0, 7, 0): 9500.968}
3698 161 0 {(2, 115, 0): 4584.5740000000005}
3698 2590 0 {(0, 7, 0): 10000.590000000002}
3698 2590 0 {(2, 115, 0): 5084.218000000001}
3699 1225 0 {(0, 7, 0): 6349.365000000001}
3699 1225 0 {(2, 115, 0): 6309.3369999999995}
3699 1570 0 {(0, 7, 0): 6494.953}
3699 1570 0 {(2, 115, 0): 6275.145}
3699 3700 0 {(0, 7, 0): 6526.723000000001}
3699 3700 0 {(2, 115, 0): 6452.503}
3700 2380 0 {(0, 7, 0): 6691.201000000001}
3700 2380 0 {(2, 115, 0): 6568.439}
3700 3699 0 {(0, 7, 0): 6516.825000000002}
3700 3699 0 {(2, 115, 0): 6442.6050000000005}
3701 2923 0 {

In [ ]:
Obj.queryR(0,3653,0,20000)
#Obj.query(0,3653,0)

In [ ]:
def modify(self,ide,u,v,n,newR):
    facility=self.G.edges[u,v,n]['Facilities']
    Rad=-1
    for i in facility:        
        if i.ident==ide:
            Rad=facility[0]
            i.R=newR
            break
    facility.sort(keys=lambda x:x.R) 
    newRad=facility[0]
    if(Rad==-1):
        print("Facility do not exist")
        return
    if newRad>Rad:
        self.preprocess(u,v,n)
    elif newRad<Rad:
        self.preprocessd(newRad,Rad,u,v,n)
def preprocessd(self,Lbound,Ubound,u,v,n):
    RTD=0
    for w in self.paths[v]:
        for z in self.G.adj[w]:
            for n1 in self.G.adj[w][z]:
                RTD=self.G.edges[w,z,n1]['length']+self.G.edges[u,v,n]['length']+self.paths[v][w]
                if u in self.paths[z].keys():
                    RTD+=self.paths[z][v]
                    if RTD>Lbound and RTD<=Ubound:
                        del self.G.edges[w,z,n1]['relevant'][(u,v,n)]

In [ ]:
facility=[]
for i in range(0,10):
    facility.append(Facility(i,10000))
print(facility)    
def search(facility,ident):
    for i in facility:
        if i.ident==ident:
            return i
    return Facility(-1,0)

if(search(facility,12).ident!=):
    print('Not Found')


In [ ]:
for i in facility:
    if i.id==5:
        break
        

In [ ]:
print(Obj.G.edges[0,7,0])

In [ ]:
Obj.add_facility(0,3653,0,28000,30)
Obj.add_facility(0,3653,0,36000,30)
Obj.add_facility(0,3653,0,20000,30)
Obj.add_facility(2,115,0,20000,30)
Obj.add_facility(2,115,0,40000,30)


In [ ]:
sorted(dict.items(),key=lambda x:x[1],reverse=True)

In [ ]:
Obj.query(0,7,0,7)

In [ ]:
def user_register(Obj,u,v,n):
    Re=min(Obj.G.edges[u,v,n]['relevant'].values())
    for w in self.paths[v]:
        for z in self.G.adj[w]:
            for n1 in self.G.adj[w][z]:
                RTD=self.G.edges[w,z,n1]['length']+self.G.edges[u,v,n]['length']+self.paths[v][w]
                if u in self.paths[z].keys():
                    RTD+=self.paths[z][v]
                    if RTD<Re and (u,v,n) not in self.G.edges[w,z,n1]['relevant'].keys():
                        self.G.edges[w,z,n1]['relevantU']+=1

In [ ]:
def user_register(Obj,u,v,n,no):
    Obj.G.edges[u,v,n]['Users']+=no
    Re=min(Obj.G.edges[u,v,n]['relevant'].values())
    for w in Obj.paths[v]:
        for z in Obj.G.adj[w]:
            for n1 in Obj.G.adj[w][z]:
                RTD=Obj.G.edges[w,z,n1]['length']+Obj.G.edges[u,v,n]['length']+Obj.paths[v][w]
                if u in Obj.paths[z].keys():
                    RTD+=Obj.paths[z][v]
                    if RTD<Re and (u,v,n): 
                        Obj.G.edges[w,z,n1]['relevantU']+=no
def user_dereister(Obj,u,v,n,no):
    Obj.G.edges[u,v,n]['Users']-=no
    Re=min(Obj.G.edges[u,v,n]['relevant'].values())
    for w in Obj.paths[v]:
        for z in Obj.G.adj[w]:
            for n1 in Obj.G.adj[w][z]:
                RTD=Obj.G.edges[w,z,n1]['length']+Obj.G.edges[u,v,n]['length']+Obj.paths[v][w]
                if u in Obj.paths[z].keys():
                    RTD+=Obj.paths[z][v]
                    if RTD<Re and (u,v,n): 
                        Obj.G.edges[w,z,n1]['relevantU']+=no
                        
def user_query(Obj,u,v,n):
    return Obj.G.edges[u,v,n]['relevantU']

In [ ]:
user_register(Obj,0,7,0)

In [ ]:
user_query(Obj,0,3653,0)